In [1]:
import json
from collections import defaultdict


In [2]:
with open("datasets_study_final.json", "r") as f:
    data = json.load(f) 

analysis_data = []
prompt_data = []    

dataset_name = defaultdict(list)
for item in data:
    if item['model']=='datasets_study.analysis':
        analysis_data.append(item)
    elif item['model']=='datasets_study.prompt':
        prompt_data.append(item)
        dataset_name[item['fields']['source_dataset']].append(item['pk'])




len(analysis_data),len(prompt_data)
print(len(dataset_name.keys()))

20


In [3]:
problem_list = {}
for item in data:
    if item['model']=='datasets_study.problem':
       problem_list[item['pk']] = item['fields']['description']
print(problem_list)

{1: 'Spelling and grammatical errors', 2: 'Very short sentence', 3: 'Partial or incomplete sentence', 4: 'Self-Admitted Technical Debt', 5: 'Automatically generated code/comments', 6: "Method name mismatches with the function/method's intent.", 7: 'Not using JavaDoc (Java) or docstring (Python) on the prompt.', 8: 'Inconsistent prompt style', 9: 'URL or reference in the comment', 12: 'Interrogation questions in the prompt', 13: 'JavaDoc/docstring has formatting issues', 14: 'Comment includes PII(author name, email address, etc.)', 15: 'The prompt description is unclear', 16: 'Incorrect input/output pair example'}


In [4]:
users = defaultdict(list)
analysis = defaultdict(list)
problems = defaultdict(list)
dataset_data = defaultdict(list)

problem_ids = defaultdict(list)


for item in analysis_data:
    # if item['fields']['user_id'] != 4:
    #     continue
    
    users[item['fields']['user_id']].append(item['fields'])
    analysis[item['fields']['prompt']].append(item['fields'])
    for problem in item['fields']['problems']:

        problems[problem].append(item['fields'])
        if problem == 13:
            problem = 7
        problem_ids[problem].append(item['fields']['prompt'])

    for data in dataset_name.keys():
        if item['fields']['prompt'] in dataset_name[data]:
            dataset_data[data].append(item['fields'])
            break
print(users.keys())
print(len(analysis))
print(len(problems))    

total = 0
for data in dataset_data.keys():
    # print(data, len(dataset_data[data]))
    if len(dataset_name[data]) == len(dataset_data[data]):
        print(data, len(dataset_name[data]), len(dataset_data[data]))


dict_keys([4, 3, 1, 11])
4164
12
HumanEval-Infilling/HumanEval-MultiLineInfilling 598 598


In [5]:
problem_ids[16]

[12,
 84,
 123,
 1130,
 1159,
 1167,
 402,
 536,
 576,
 582,
 594,
 638,
 648,
 673,
 687,
 693,
 708,
 721,
 728,
 729,
 750,
 227,
 240,
 273,
 303,
 351,
 771,
 778,
 779,
 792,
 796,
 812,
 823,
 835,
 839,
 854,
 865,
 898,
 906,
 924,
 925,
 936,
 12,
 123,
 227,
 240,
 273,
 351,
 402,
 536,
 576,
 638,
 925,
 936,
 1064,
 1145,
 1159]

In [6]:
problems = sorted(problems.items(), key=lambda x: x[0])
# print(problems)

In [7]:
for problem in problem_ids:
    print(problem, len(problem_ids[problem]), len(set(problem_ids[problem])))

1 1122 624
8 911 342
7 7453 2600
15 326 189
16 57 44
6 3962 1353
4 1 1
9 54 18
3 339 141
12 445 155
2 1 1


In [8]:
for item in problems:
    key = item[0]
    total = len(item[1])
    print(f"{key}. {problem_list[key]}: {total}")


1. Spelling and grammatical errors: 1122
2. Very short sentence: 1
3. Partial or incomplete sentence: 339
4. Self-Admitted Technical Debt: 1
6. Method name mismatches with the function/method's intent.: 3962
7. Not using JavaDoc (Java) or docstring (Python) on the prompt.: 1313
8. Inconsistent prompt style: 911
9. URL or reference in the comment: 54
12. Interrogation questions in the prompt: 445
13. JavaDoc/docstring has formatting issues: 6140
15. The prompt description is unclear: 326
16. Incorrect input/output pair example: 57


In [9]:
merged_analysis = defaultdict(list)
for item in analysis_data:
    if item['fields']['user_id'] != 3:
        continue
    merged_analysis[item['fields']['prompt']].append(item['fields'])


In [10]:
user_3_analysis = defaultdict(str)
user_4_analysis = defaultdict(str)
for item in users[3]:
    item['problems'] = '#'.join(sorted( [str(x) for x in item['problems']] ))
    user_3_analysis[item['prompt']] = item['problems']


for item in users[4]:
    item['problems'] = '#'.join(sorted( [str(x) for x in item['problems']] ))
    user_4_analysis[item['prompt']] = item['problems']

len(user_3_analysis), len(user_4_analysis)
print(user_3_analysis)

defaultdict(<class 'str'>, {70: '13#8', 71: '1#13#8', 72: '1#6#8', 73: '1#13#8', 74: '1#13#8', 75: '1#13#8', 76: '1#13#8', 78: '1#13#8', 77: '1#13#8', 79: '1#13#6#7#8', 80: '1#13#8', 81: '13#6#8', 82: '1#13#6#8', 83: '1#13#8', 84: '13#8', 85: '13#7#8', 86: '1#13#8', 87: '13#8', 88: '13#8', 89: '1#13#8', 90: '1#13#6#8', 91: '1#13#8', 92: '1#13#8', 93: '1#13#8', 1058: '1#13', 1059: '1#13#8', 1060: '1#13#8', 1061: '1#13#8', 1062: '1#13#8', 1063: '1#13#8', 1064: '1#13#8', 1065: '1#13#8', 1066: '1#13', 1067: '13#8', 1068: '1#13#8', 1069: '1#13', 1070: '1#13', 1071: '1#13#8', 1072: '1#13', 1073: '1#13#8', 1074: '13', 1075: '1#13#8', 1076: '1#13#15#8', 1077: '1#13', 1078: '1#13#8', 1079: '1#13#8', 1080: '1#13#8', 1081: '13#8', 1082: '13#8', 1083: '1#13', 1084: '1#13', 1085: '1#13', 1086: '1#13', 1087: '13', 1088: '13', 1089: '1#13#8', 1090: '13#8', 1091: '13', 1092: '1#13', 1093: '1#13', 1094: '1#13', 1095: '13', 1096: '13', 1097: '13', 1098: '13', 1099: '1#13', 1100: '1#13', 1101: '1#13', 11

In [11]:
len(merged_analysis.keys())

3566

In [12]:
user_3 = []
user_4 = []
problem_wise = defaultdict(list)
for key in merged_analysis.keys():
    if key not in user_3_analysis:
        user_3_analysis[key] = ''
    if key not in user_4_analysis:
        user_4_analysis[key] = ''
    user_3.append(user_3_analysis[key])
    user_4.append(user_4_analysis[key])

    for i in range(1,17):
        if str(i) in user_3_analysis[key].split('#'):
            problem_wise[i].append(1)
        else:
            problem_wise[i].append(0)
            
        if str(i) in user_4_analysis[key].split('#'):
            problem_wise[i].append(1)
        else:
            problem_wise[i].append(0)
print(user_3)

['13#8', '1#13#8', '1#6#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#6#7#8', '1#13#8', '13#6#8', '1#13#6#8', '1#13#8', '13#8', '13#7#8', '1#13#8', '13#8', '13#8', '1#13#8', '1#13#6#8', '1#13#8', '1#13#8', '1#13#8', '1#13', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13', '13#8', '1#13#8', '1#13', '1#13', '1#13#8', '1#13', '1#13#8', '13', '1#13#8', '1#13#15#8', '1#13', '1#13#8', '1#13#8', '1#13#8', '13#8', '13#8', '1#13', '1#13', '1#13', '1#13', '13', '13', '1#13#8', '13#8', '13', '1#13', '1#13', '1#13', '13', '13', '13', '13', '1#13', '1#13', '1#13', '1#13#8', '13', '1#13', '1#13', '1#13', '1#13', '1#13', '13', '1#13', '1#13#8', '1#13', '1#13#8', '1#13', '1#13', '13', '1#13#8', '1#13#8', '13', '1#13', '1#13#8', '13', '13', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '13#8', '13#8', '1#13#8', '13#8', '13#6#8', '1#13#8', '13#8', '1#13#8', '1#13#8', '

In [13]:
print(len(user_3), len(user_4))
print(user_4)

3566 3566
['13#8', '13#8', '6#8', '13#8', '13#8', '1#13#15', '13#15#8', '13#8', '13#8', '13#6#8', '13#8', '13#6#8', '1#13#8', '13#8', '13#16#8', '13#8', '13#8', '13#8', '13#8', '13#8', '13#6#8', '13#8', '13#6#8', '13#6', '13', '13#8', '1#13#8', '13#8', '13#8', '1#13#8', '13#8', '13#8', '13', '13#8', '13#8', '13', '13', '1#13#8', '1#13', '1#13#8', '13', '13#8', '13#8', '13', '13#8', '13#8', '13#8', '13#8', '13#8', '13', '1#13', '13', '1#13', '13', '13', '13#8', '13#8', '13', '13', '1#13', '13', '13', '13', '13', '13', '13', '13', '1#13', '13#8', '13', '13', '13#8', '1#13', '13', '13', '1#13', '13', '13#8', '13#6', '13#8', '13', '13', '13', '13#8', '1#13#8', '13', '1#13', '13#8', '13', '13', '13#8', '1#13#8', '13#8', '13#8', '13#8', '13#8', '13#8', '13#8', '13#8', '13#8', '1#13#8', '13#8', '13#15#8', '1#13#8', '13#6#8', '13#8', '13#8', '13#6#8', '1#13#8', '13#8', '13#8', '13#8', '1#8', '13#8', '1#8', '13#8', '13#8', '13#8', '1#13#6#8', '13#8', '13#8', '13#6#8', '13#8', '13#8', '13#8', '1

In [14]:
from sklearn.metrics import cohen_kappa_score
print(user_3)
cohen_kappa_score(user_3, user_4)

['13#8', '1#13#8', '1#6#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#6#7#8', '1#13#8', '13#6#8', '1#13#6#8', '1#13#8', '13#8', '13#7#8', '1#13#8', '13#8', '13#8', '1#13#8', '1#13#6#8', '1#13#8', '1#13#8', '1#13#8', '1#13', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13', '13#8', '1#13#8', '1#13', '1#13', '1#13#8', '1#13', '1#13#8', '13', '1#13#8', '1#13#15#8', '1#13', '1#13#8', '1#13#8', '1#13#8', '13#8', '13#8', '1#13', '1#13', '1#13', '1#13', '13', '13', '1#13#8', '13#8', '13', '1#13', '1#13', '1#13', '13', '13', '13', '13', '1#13', '1#13', '1#13', '1#13#8', '13', '1#13', '1#13', '1#13', '1#13', '1#13', '13', '1#13', '1#13#8', '1#13', '1#13#8', '1#13', '1#13', '13', '1#13#8', '1#13#8', '13', '1#13', '1#13#8', '13', '13', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '1#13#8', '13#8', '13#8', '1#13#8', '13#8', '13#6#8', '1#13#8', '13#8', '1#13#8', '1#13#8', '

0.7564578065084819

In [15]:
for i in range(1,17):
    user_3 = []
    user_4 = []
    # print(problem_wise[i])
    for index, item in enumerate(problem_wise[i]):
        if index%2==0:
            user_3.append(item)
        else:
            user_4.append(item)
    print(sum(user_3), sum(user_4))
    print(f"Problem {i}: {cohen_kappa_score(user_3, user_4)}")

472 347
Problem 1: 0.4182699019585914
1 0
Problem 2: 0.0
84 123
Problem 3: 0.6868869320890093
1 0
Problem 4: 0.0
0 0
Problem 5: nan
1317 1324
Problem 6: 0.9609148808088507
435 439
Problem 7: 0.9869604359295205
297 302
Problem 8: 0.8450869498110543
18 18
Problem 9: 1.0
0 0
Problem 10: nan
0 0
Problem 11: nan
147 146
Problem 12: 0.9394938088498613
2157 2148
Problem 13: 0.9923817918943107
0 0
Problem 14: nan
119 83
Problem 15: 0.2873884135670832
5 37
Problem 16: -0.002476539805357225


/Users/lsiddiqsunny/.pyenv/versions/3.9.4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/lsiddiqsunny/.pyenv/versions/3.9.4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/lsiddiqsunny/.pyenv/versions/3.9.4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/lsiddiqsunny/.pyenv/versions/3.9.4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
